<a href="https://colab.research.google.com/github/yeoauqt/229352/blob/main/Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [26]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [27]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [28]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [29]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'yes': 1, 'no': 0}) # Write your code here


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
X = df.drop(['y', 'y_new'], axis=1)

# Split the data BEFORE any transformations


# Print data shape
print(y)
print(X)


0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y_new, Length: 41188, dtype: int64
       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          high.school      no      n

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
33977,36,blue-collar,married,basic.4y,NaN,yes,no,cellular,may,wed,300,1,999,1,failure
9283,32,housemaid,married,high.school,no,yes,yes,telephone,jun,fri,292,2,999,0,nonexistent
24619,40,entrepreneur,divorced,university.degree,no,yes,no,cellular,nov,mon,124,2,999,0,nonexistent
38460,31,self-employed,single,university.degree,no,no,no,cellular,oct,fri,76,1,999,0,nonexistent
12442,42,housemaid,divorced,basic.4y,NaN,no,no,cellular,jul,mon,461,2,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16673,50,management,married,university.degree,no,yes,no,cellular,jul,wed,110,2,999,0,nonexistent
23837,51,services,married,high.school,no,yes,no,cellular,aug,fri,179,1,999,0,nonexistent
33888,29,admin.,single,university.degree,no,yes,no,cellular,may,wed,152,1,999,1,failure
36392,29,admin.,single,university.degree,no,yes,no,telephone,jun,tue,128,1,999,0,nonexistent


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [7]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training data
X_train[num_cols_demo] = scaler.fit_transform(X_train[num_cols_demo])
X_test[num_cols_demo] = scaler.transform(X_test[num_cols_demo])

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
20992,-0.572166,technician,married,high.school,no,yes,no,cellular,aug,thu,85,0.165333,999,0,nonexistent
38381,2.404086,retired,married,NaN,no,NaN,NaN,telephone,oct,mon,208,0.165333,999,1,failure
39693,0.675940,services,divorced,university.degree,no,yes,yes,cellular,may,tue,336,-0.572779,15,3,failure
30090,-1.244223,management,married,university.degree,no,no,no,cellular,apr,mon,54,-0.572779,999,0,nonexistent
19715,-0.572166,admin.,single,university.degree,no,yes,no,cellular,aug,thu,259,0.165333,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12620,0.291907,blue-collar,married,basic.9y,no,yes,no,cellular,jul,mon,107,1.272501,999,0,nonexistent
6596,-0.668174,blue-collar,married,basic.6y,no,yes,no,telephone,may,wed,289,-0.572779,999,0,nonexistent
690,0.003883,admin.,divorced,university.degree,no,yes,no,telephone,may,tue,189,0.165333,999,0,nonexistent
2465,1.828038,retired,married,university.degree,no,no,no,telephone,may,tue,916,-0.203723,999,0,nonexistent


Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [8]:
X_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,5.446566e-17,259.694530,3.302443e-17,962.005029,0.172557
std,1.000017e+00,259.215231,1.000017e+00,188.077713,0.495016
min,-2.204305e+00,0.000000,-5.727786e-01,0.000000,0.000000
25%,-7.641824e-01,103.000000,-5.727786e-01,999.000000,0.000000
50%,-1.881335e-01,181.000000,-2.037226e-01,999.000000,0.000000
75%,6.759398e-01,322.000000,1.653334e-01,999.000000,0.000000
max,5.572355e+00,4918.000000,1.492757e+01,999.000000,7.000000


**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [9]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

## Write your code here
X_train[ord_col_demo] = imputer_ord.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = imputer_ord.transform(X_test[ord_col_demo])

X_train['education']

,education
36634,high.school
13692,basic.9y
848,professional.course
24096,basic.9y
28291,basic.6y
...,...
18373,professional.course
7473,high.school
24426,basic.9y
4765,university.degree


- **Ordinal Encoding**

In [10]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [11]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

## Write your code here
X_train[ord_col_demo] = ordinal_encoder.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = ordinal_encoder.transform(X_test[ord_col_demo])

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
20992,-0.572166,technician,married,4.0,no,yes,no,cellular,aug,thu,85,0.165333,999,0,nonexistent
38381,2.404086,retired,married,6.0,no,NaN,NaN,telephone,oct,mon,208,0.165333,999,1,failure
39693,0.675940,services,divorced,6.0,no,yes,yes,cellular,may,tue,336,-0.572779,15,3,failure
30090,-1.244223,management,married,6.0,no,no,no,cellular,apr,mon,54,-0.572779,999,0,nonexistent
19715,-0.572166,admin.,single,6.0,no,yes,no,cellular,aug,thu,259,0.165333,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12620,0.291907,blue-collar,married,3.0,no,yes,no,cellular,jul,mon,107,1.272501,999,0,nonexistent
6596,-0.668174,blue-collar,married,2.0,no,yes,no,telephone,may,wed,289,-0.572779,999,0,nonexistent
690,0.003883,admin.,divorced,6.0,no,yes,no,telephone,may,tue,189,0.165333,999,0,nonexistent
2465,1.828038,retired,married,6.0,no,no,no,telephone,may,tue,916,-0.203723,999,0,nonexistent


Let's take a look at the imputed and ordinal-encoded `education`.

In [ ]:
print("\nOriginal X_train 'education' head:")
print(X_train[ord_col_demo].iloc[20:25])
print("\nImputed X_train 'education' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_ord_demo, columns=ord_col_demo, index=X_train.index).iloc[20:25])
print("\nOrdinal Encoded X_train 'education' head:")
print(pd.DataFrame(X_train_ord_encoded_demo, columns=ord_col_demo, index=X_train.index).iloc[20:25])

**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [12]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(X_train[nom_col_demo])

X_train[nom_col_demo] = imputer_nom.fit_transform(X_train[nom_col_demo])
X_test[nom_col_demo] = imputer_nom.transform(X_test[nom_col_demo])

- **Nominal Encoding**

In [13]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

## Write your code here
X_train_onehot = onehot_encoder.fit_transform(X_train[['job']])
X_test_onehot = onehot_encoder.transform(X_test[['job']])

X_train_onehot.shape

(28831, 11)

In [14]:
X_train_onehot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [15]:
X_train = pd.concat([X_train.reset_index(drop=True),
                     pd.DataFrame(X_train_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

X_train

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-1.436239,services,divorced,4.0,no,yes,yes,cellular,jun,thu,437,-0.203723,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.860191,blue-collar,single,3.0,no,yes,no,cellular,jul,thu,486,-0.572779,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.003883,technician,married,5.0,no,yes,no,telephone,may,wed,327,0.165333,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.732029,retired,divorced,3.0,no,yes,no,telephone,nov,mon,248,-0.572779,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.579932,blue-collar,divorced,2.0,NaN,no,no,cellular,apr,wed,179,0.903445,999,2,failure,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,-0.476158,technician,married,5.0,no,NaN,NaN,cellular,jul,thu,12,3.486837,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
28827,1.924046,admin.,married,4.0,no,yes,no,telephone,may,fri,118,-0.203723,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28828,-0.380150,blue-collar,married,3.0,NaN,yes,no,cellular,nov,mon,499,0.165333,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28829,0.771948,admin.,married,6.0,no,yes,no,telephone,may,wed,396,-0.572779,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
X_test = pd.concat([X_test.reset_index(drop=True),
                     pd.DataFrame(X_test_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-0.572166,technician,married,4.0,no,yes,no,cellular,aug,thu,85,0.165333,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2.404086,retired,married,6.0,no,NaN,NaN,telephone,oct,mon,208,0.165333,999,1,failure,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.675940,services,divorced,6.0,no,yes,yes,cellular,may,tue,336,-0.572779,15,3,failure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-1.244223,management,married,6.0,no,no,no,cellular,apr,mon,54,-0.572779,999,0,nonexistent,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.572166,admin.,single,6.0,no,yes,no,cellular,aug,thu,259,0.165333,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,0.291907,blue-collar,married,3.0,no,yes,no,cellular,jul,mon,107,1.272501,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12353,-0.668174,blue-collar,married,2.0,no,yes,no,telephone,may,wed,289,-0.572779,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12354,0.003883,admin.,divorced,6.0,no,yes,no,telephone,may,tue,189,0.165333,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12355,1.828038,retired,married,6.0,no,no,no,telephone,may,tue,916,-0.203723,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print("\nOriginal X_train 'job' head:")
print(X_train[nom_col_demo].iloc[40:45])
print("\nImputed X_train 'job' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_nom_demo, columns=nom_col_demo, index=X_train.index).iloc[40:45])
print("\nOne-Hot Encoded X_train 'job' shape:", X_train_onehot_encoded_demo.shape)
print("First 5 rows of One-Hot Encoded X_train 'job':")
print(pd.DataFrame(X_train_onehot_encoded_demo, columns=onehot_encoder.get_feature_names_out(nom_col_demo), index=X_train.index).iloc[40:45])

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [31]:
# Make lists for numerical_features, ordinal_features, and nominal_features
num_cols = ['age', 'campaign', 'duration', 'pdays', 'previous']
ord_cols = ['education']
nom_cols = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

In [32]:
# make a copy of X_train and X_test
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

**1. Numerical Feature: age, campaign, duration, pdays  and previous (Standard Scaling)**

In [33]:
df['pdays'] = df['pdays'].replace(-1, 0)

In [34]:
num_cols = ['age', 'campaign', 'duration', 'pdays', 'previous']

scaler = StandardScaler()

X_train_copy[num_cols] = scaler.fit_transform(X_train_copy[num_cols])
X_test_copy[num_cols] = scaler.transform(X_test_copy[num_cols])

X_test_copy

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
33977,-0.387075,blue-collar,married,basic.4y,NaN,yes,no,cellular,may,wed,0.160935,-0.566815,0.197179,1.642122,failure
9283,-0.772337,housemaid,married,high.school,no,yes,yes,telephone,jun,fri,0.130109,-0.209233,0.197179,-0.349298,nonexistent
24619,-0.001814,entrepreneur,divorced,university.degree,no,yes,no,cellular,nov,mon,-0.517236,-0.209233,0.197179,-0.349298,nonexistent
38460,-0.868652,self-employed,single,university.degree,no,no,no,cellular,oct,fri,-0.702192,-0.566815,0.197179,-0.349298,nonexistent
12442,0.190817,housemaid,divorced,basic.4y,NaN,no,no,cellular,jul,mon,0.781306,-0.209233,0.197179,-0.349298,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16673,0.961339,management,married,university.degree,no,yes,no,cellular,jul,wed,-0.571181,-0.209233,0.197179,-0.349298,nonexistent
23837,1.057655,services,married,high.school,no,yes,no,cellular,aug,fri,-0.305308,-0.566815,0.197179,-0.349298,nonexistent
33888,-1.061283,admin.,single,university.degree,no,yes,no,cellular,may,wed,-0.409345,-0.566815,0.197179,1.642122,failure
36392,-1.061283,admin.,single,university.degree,no,yes,no,telephone,jun,tue,-0.501823,-0.566815,0.197179,-0.349298,nonexistent


In [35]:
X_train_copy.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,2.883100e+04,2.883100e+04,2.883100e+04,2.883100e+04
mean,1.516906e-16,-4.707213e-17,-4.929019e-17,-1.842221e-16,-1.626576e-17
std,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00
min,-2.217067e+00,-9.950380e-01,-5.668147e-01,-5.103598e+00,-3.492980e-01
25%,-7.723367e-01,-6.020073e-01,-5.668147e-01,1.971789e-01,-3.492980e-01
50%,-1.944447e-01,-3.053077e-01,-2.092331e-01,1.971789e-01,-3.492980e-01
75%,6.723934e-01,2.341462e-01,1.483484e-01,1.971789e-01,-3.492980e-01
max,5.584475e+00,1.518472e+01,1.910017e+01,1.971789e-01,1.359064e+01


**2. Ordinal Feature: education (Ordinal Encoding with Imputation)**

In [36]:
ord_cols = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')


X_train_copy[ord_cols] = imputer_ord.fit_transform(X_train_copy[ord_cols])
X_test_copy[ord_cols] = imputer_ord.transform(X_test_copy[ord_cols])

X_train_copy['education']

,education
12124,high.school
39433,high.school
39197,university.degree
27280,basic.4y
32790,basic.9y
...,...
23124,basic.4y
32775,basic.9y
20002,university.degree
9256,university.degree


In [37]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [38]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

X_train_copy[ord_cols] = ordinal_encoder.fit_transform(X_train_copy[ord_cols])
X_test_copy[ord_cols] = ordinal_encoder.transform(X_test_copy[ord_cols])

X_test_copy

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
33977,-0.387075,blue-collar,married,1.0,NaN,yes,no,cellular,may,wed,0.160935,-0.566815,0.197179,1.642122,failure
9283,-0.772337,housemaid,married,4.0,no,yes,yes,telephone,jun,fri,0.130109,-0.209233,0.197179,-0.349298,nonexistent
24619,-0.001814,entrepreneur,divorced,6.0,no,yes,no,cellular,nov,mon,-0.517236,-0.209233,0.197179,-0.349298,nonexistent
38460,-0.868652,self-employed,single,6.0,no,no,no,cellular,oct,fri,-0.702192,-0.566815,0.197179,-0.349298,nonexistent
12442,0.190817,housemaid,divorced,1.0,NaN,no,no,cellular,jul,mon,0.781306,-0.209233,0.197179,-0.349298,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16673,0.961339,management,married,6.0,no,yes,no,cellular,jul,wed,-0.571181,-0.209233,0.197179,-0.349298,nonexistent
23837,1.057655,services,married,4.0,no,yes,no,cellular,aug,fri,-0.305308,-0.566815,0.197179,-0.349298,nonexistent
33888,-1.061283,admin.,single,6.0,no,yes,no,cellular,may,wed,-0.409345,-0.566815,0.197179,1.642122,failure
36392,-1.061283,admin.,single,6.0,no,yes,no,telephone,jun,tue,-0.501823,-0.566815,0.197179,-0.349298,nonexistent


**3. Nominal Feature: job, marital, default, housing, loan, contact, month, day_of_week and poutcome (One-Hot Encoding with Imputation)**

In [39]:
nom_cols = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(X_train_copy[nom_cols])

X_train_copy[nom_cols] = imputer_nom.fit_transform(X_train_copy[nom_cols])
X_test_copy[nom_cols] = imputer_nom.transform(X_test_copy[nom_cols])

In [40]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_copy_onehot = onehot_encoder.fit_transform(X_train_copy[nom_cols])
X_test_copy_onehot = onehot_encoder.transform(X_test_copy[nom_cols])

X_train_copy_onehot.shape

(28831, 40)

In [41]:
X_train_copy = pd.concat([X_train_copy.reset_index(drop=True),
                     pd.DataFrame(X_train_copy_onehot, columns=onehot_encoder.get_feature_names_out(nom_cols))], axis=1)

X_train_copy

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,-0.098129,admin.,single,4.0,no,no,yes,telephone,jun,mon,-0.386226,-0.566815,0.197179,-0.349298,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.865024,admin.,married,4.0,no,no,no,cellular,apr,mon,-0.536502,-0.566815,-5.071762,1.642122,success,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.094501,admin.,divorced,6.0,no,yes,no,cellular,mar,fri,0.453781,-0.209233,0.197179,-0.349298,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.250285,blue-collar,married,1.0,no,yes,no,cellular,nov,fri,-0.301454,-0.566815,0.197179,-0.349298,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-1.253913,blue-collar,married,3.0,no,no,no,cellular,may,mon,1.521129,-0.566815,0.197179,-0.349298,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,0.672393,blue-collar,married,1.0,no,no,no,cellular,aug,tue,-0.686779,-0.209233,0.197179,-0.349298,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
28827,-0.579706,services,divorced,3.0,no,yes,no,cellular,may,mon,0.796719,-0.566815,0.197179,-0.349298,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
28828,-0.194445,entrepreneur,married,6.0,no,yes,no,cellular,aug,fri,-0.270629,0.863511,0.197179,-0.349298,nonexistent,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
28829,0.768709,self-employed,married,6.0,no,no,no,telephone,jun,fri,0.804426,0.148348,0.197179,-0.349298,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [42]:
X_test_copy = pd.concat([X_test_copy.reset_index(drop=True),
                     pd.DataFrame(X_test_copy_onehot, columns=onehot_encoder.get_feature_names_out(nom_cols))], axis=1)

X_test_copy

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,-0.387075,blue-collar,married,1.0,no,yes,no,cellular,may,wed,0.160935,-0.566815,0.197179,1.642122,failure,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,-0.772337,housemaid,married,4.0,no,yes,yes,telephone,jun,fri,0.130109,-0.209233,0.197179,-0.349298,nonexistent,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.001814,entrepreneur,divorced,6.0,no,yes,no,cellular,nov,mon,-0.517236,-0.209233,0.197179,-0.349298,nonexistent,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.868652,self-employed,single,6.0,no,no,no,cellular,oct,fri,-0.702192,-0.566815,0.197179,-0.349298,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.190817,housemaid,divorced,1.0,no,no,no,cellular,jul,mon,0.781306,-0.209233,0.197179,-0.349298,nonexistent,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,0.961339,management,married,6.0,no,yes,no,cellular,jul,wed,-0.571181,-0.209233,0.197179,-0.349298,nonexistent,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
12353,1.057655,services,married,4.0,no,yes,no,cellular,aug,fri,-0.305308,-0.566815,0.197179,-0.349298,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12354,-1.061283,admin.,single,6.0,no,yes,no,cellular,may,wed,-0.409345,-0.566815,0.197179,1.642122,failure,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
12355,-1.061283,admin.,single,6.0,no,yes,no,telephone,jun,tue,-0.501823,-0.566815,0.197179,-0.349298,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


**Train Logistic Regression on the preprocessed X_train_copy and y_train**

In [47]:
X_train_copy = X_train_copy.drop(columns=nom_cols)
X_test_copy = X_test_copy.drop(columns=nom_cols)

In [48]:
model = LogisticRegression()

model.fit(X_train_copy,y_train)

LogisticRegression()

In [49]:
y_pred_train = model.predict(X_train_copy)

**Evaluate the Model: Make predictions on the preprocessed X_test_copy**

In [50]:
y_pred = model.predict(X_test_copy)

In [52]:
# classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95     11027
           1       0.63      0.36      0.46      1330

    accuracy                           0.91     12357
   macro avg       0.78      0.67      0.70     12357
weighted avg       0.89      0.91      0.90     12357



## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [ ]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


